In [65]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
from tqdm import tqdm

In [66]:
frequency = 'monthly'
column = 'ridership'
years = [2020,2021,2022,2023]

In [67]:
nyc_master_df = pd.read_csv(f'Data/Master Sets/NYC Master/Yearly Combined/NYC_Combined_{frequency}.csv')

In [ ]:
nyc_master_df.head()

In [69]:
nyc_master_df = nyc_master_df[nyc_master_df['year'].isin(years)]

In [70]:
station_network_list = nyc_master_df['GTFS Stop ID'].unique()
station_network_list.sort()
time_interval = nyc_master_df['unique_time'].unique()
time_interval.sort()

In [71]:
def compute_time_series(col: str):

    if col not in nyc_master_df:
        raise ValueError('Column does not exist')
    temp_dict = {}

    if is_numeric_dtype(nyc_master_df[col]):
        print('is numeric')
    else:
        print('is not numeric')

    for station in tqdm(station_network_list):
        for time in time_interval:
            temp_df = nyc_master_df[(nyc_master_df['GTFS Stop ID'] == station) & (nyc_master_df['unique_time'] == time)]
            if is_numeric_dtype(temp_df[col]):
                val = temp_df[col].mean() # Takes mean of all rows
            else:
                val = temp_df[col].max()
            if station not in temp_dict:
                temp_dict[station] = {}
            temp_dict[station][time] = val
    return temp_dict

In [ ]:
ts_matrix = pd.DataFrame(compute_time_series(column))

In [ ]:
ts_matrix

In [74]:
# Shortcut solution to set all NA values to 0
ts_matrix.fillna(0, inplace=True)

# TODO Imputation for NA values

In [75]:
ts_matrix.to_csv(f'Data/Master Sets/NYC Master/NYC_Time_Series_{frequency}_{column}.csv')